In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import fitz
from io import BytesIO
from PIL import Image
import base64
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import uuid
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
import json
from langchain.prompts import PromptTemplate

In [ ]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
openai_ef = ???
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
loader = PyPDFLoader("./data/cpi.pdf")
pdf_pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pdf_pages)

In [ ]:
db = ???
collection = ???

In [ ]:
ids = []
texts = []
metadatas = []
for doc in docs:
    texts.append(doc.page_content)
    metadata = doc.metadata
    metadata["type"] = "text"
    metadatas.append(metadata)
    ids.append(str(uuid.uuid1()))

In [ ]:
collection.add(
    documents=texts,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
res = collection.query(
    query_texts=["inflation and house pricing"],
    n_results=3
)
print(res)

In [ ]:
## retrieve images

In [ ]:
## summarize image
def image_summarize(img_base64, prompt):

In [ ]:
prompt = """You are an assistant tasked with summarizing images for retrieval. \
These summaries will be embedded and used to retrieve the raw image. \
Give a concise summary of the image that is well optimized for retrieval."""

In [ ]:
## iterate images and store in db

In [ ]:
res = collection.query(
    query_texts=["percentage distribution of consumer"],
    n_results=1
)
print(res)

In [ ]:
print(collection.get())

In [ ]:
template = """
Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. Include in the answer the metadata and the id. Example: 
        Answer: here put the answer
        Id: here put the context id
        Metadata: here put the metadata
2. If you don't know the answer, don't try to make up an answer. Just say "I don't know" as the answer and None as the metadata
3. If you find the answer, write the answer in a concise way.

{context}

Question: {question}
Answer:"""

In [ ]:
query = "What percentage of CPI is dedicated to Housing, and how does it compare to the combined percentage of Medical Care, Apparel, and Other Goods and Services?"
db_result = res = collection.query(
    query_texts=[query],
    n_results=3
)
print(db_result)

In [ ]:
context = ""
for i in range(len(db_result['ids'][0])):
    current = "Id:" + db_result['ids'][0][i]
    current += "\nContext: " + db_result["documents"][0][i]
    metadata_str = json.dumps(db_result["metadatas"][0][i])
    current += "\nMetadata: " + metadata_str
    current += "\n\n"
    context += current
print(context)

In [ ]:
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [ ]:
str = prompt.format(context=context, question=query)
print(str)

In [ ]:
llm = ChatOpenAI(
  model="gpt-3.5-turbo",
  temperature=0,
  api_key=OPENAI_API_KEY
)
res = llm.invoke(str)
print(res)